
# 02 — Data Wrangling (SpaceX Falcon 9)

**Upstream inputs (from `01_data-collection-api`):**
- `data/launches_exploded.csv` (one row per payload per launch)
- `data/launches_summary.csv` (per-launch aggregate)

**Goals in this notebook:**
1. Clean and standardize columns (types, missing values, categorical normalizations).
2. Enrich with **booster version labels** (e.g., `F9 v1.1`, `FT`, `Block 5`) using Wikipedia — with robust fallbacks.
3. Engineer analysis-ready fields required by the rubric (e.g., `landing_outcome_kind`, `is_nasa`).
4. Save a single tidy dataset for SQL, Folium, Dash, and ML steps.

**Outputs produced:**
- `data/launches_clean.csv` — analysis-ready, one row per payload per launch.
- `data/booster_version_lookup.csv` — mapping for traceability.


## Setup

In [4]:

# Uncomment if running in a fresh env:
# !pip install pandas numpy requests lxml beautifulsoup4

import os, re, json, time, math, pathlib, warnings
from pathlib import Path
from typing import Dict, Any, List, Optional
import pandas as pd
import numpy as np
import requests

pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 180)

DATA_DIR = Path("./data")
ARTIFACTS_DIR = Path("./artifacts")
DATA_DIR.mkdir(exist_ok=True, parents=True)
ARTIFACTS_DIR.mkdir(exist_ok=True, parents=True)

EXPL_PATH = DATA_DIR / "launches_exploded.csv"
SUMM_PATH = DATA_DIR / "launches_summary.csv"

assert EXPL_PATH.exists(), "Missing data/launches_exploded.csv — run 01_data-collection-api first."
assert SUMM_PATH.exists(), "Missing data/launches_summary.csv — run 01_data-collection-api first."

df = pd.read_csv(EXPL_PATH)

# Coerce types
df['flight_number'] = pd.to_numeric(df['flight_number'], errors='coerce').astype('Int64')
df['payload_mass_kg'] = pd.to_numeric(df['payload_mass_kg'], errors='coerce')
df['date_utc'] = pd.to_datetime(df['date_utc'], errors='coerce', utc=True)
if 'year' in df.columns:
    df['year'] = pd.to_numeric(df['year'], errors='coerce').astype('Int64')
else:
    df['year'] = df['date_utc'].dt.year.astype('Int64')

# Clean text columns
def _norm_str(s):
    if pd.isna(s): return s
    s = str(s).strip()
    s = re.sub(r"\s+", " ", s)
    return s

for col in ['launch_site','rocket_name','orbit','customers','nationalities','landing_outcome']:
    if col in df.columns:
        df[col] = df[col].apply(_norm_str)

# Normalize orbit codes to upper-case (keep as-is otherwise)
if 'orbit' in df.columns:
    df['orbit'] = df['orbit'].str.upper()

# Keep a canonical 'launch_outcome' string and boolean
df['launch_success'] = df['launch_success'].astype('boolean')

# Derive landing_outcome_kind: drone / ground / other
def derive_outcome_kind(x: Optional[str]) -> Optional[str]:
    if not isinstance(x, str):
        return None
    xlow = x.lower()
    if "drone" in xlow:
        return "drone"
    if "ground" in xlow or "rtls" in xlow:
        return "ground"
    if "no attempt" in xlow:
        return "no_attempt"
    if "success" in xlow:
        return "success_unknown_site"
    if "failure" in xlow:
        return "failure_unknown_site"
    return "unknown"

df['landing_outcome_kind'] = df['landing_outcome'].apply(derive_outcome_kind)

# Flag NASA missions (for SQL: total payload carried by NASA boosters)
def is_nasa(customers: Optional[str]) -> bool:
    if not isinstance(customers, str):
        return False
    return bool(re.search(r"\bNASA\b", customers, flags=re.IGNORECASE))

df['is_nasa'] = df['customers'].apply(is_nasa)

print("Rows loaded:", len(df))
df.head(3)


Rows loaded: 192


,flight_number,date_utc,year,launch_site,site_region,site_locality,site_lat,site_lon,rocket_name,core_serial,booster_block,landing_outcome,launch_success,payload_id,payload_name,payload_mass_kg,orbit,customers,nationalities,landing_outcome_kind,is_nasa
0,6,2010-06-04 18:45:00+00:00,2010,CCSFS SLC 40,Florida,Cape Canaveral,28.561857,-80.577366,Falcon 9,B0003,1,No attempt,True,5eb0e4b7b6c3bb0006eeb1e7,Dragon Qualification Unit,NaN,LEO,SpaceX,United States,no_attempt,False
1,7,2010-12-08 15:43:00+00:00,2010,CCSFS SLC 40,Florida,Cape Canaveral,28.561857,-80.577366,Falcon 9,B0004,1,No attempt,True,5eb0e4b9b6c3bb0006eeb1e8,COTS Demo Flight 1,NaN,LEO,NASA(COTS),United States,no_attempt,True
2,7,2010-12-08 15:43:00+00:00,2010,CCSFS SLC 40,Florida,Cape Canaveral,28.561857,-80.577366,Falcon 9,B0004,1,No attempt,True,5eb0e4b9b6c3bb0006eeb1e9,Cubesats,NaN,LEO,NRO,NaN,no_attempt,False



## Booster version enrichment

We attempt to scrape **Wikipedia — List of Falcon 9 and Falcon Heavy launches** to get legacy booster
version labels (e.g., `F9 v1.1`, `FT`, `Block 5`). If scraping fails (offline or schema change), we
fall back to a **date-based heuristic**:

- `F9 v1.0` — before **2013-09-29**  
- `F9 v1.1` — **2013-09-29** to **2015-12-21**  
- `FT (v1.2)` — **2015-12-22** to **2018-05-10**  
- `Block 5` — **2018-05-11** and later


In [7]:

WIKI_URL = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"
UA = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                    "AppleWebKit/537.36 (KHTML, like Gecko) "
                    "Chrome/124.0 Safari/537.36"}

def fetch_booster_version_lookup() -> pd.DataFrame:
    try:
        html = requests.get(WIKI_URL, headers=UA, timeout=30)
        html.raise_for_status()
        tables = pd.read_html(html.text)  # requires lxml installed
    except Exception as e:
        print("Wikipedia fetch failed, using fallback:", e)
        return pd.DataFrame()

    # Heuristic: parse tables that contain 'Date and time (UTC)' & 'Version'/'Booster version'/ 'Vehicle' info
    dfs = []
    for t in tables:
        cols = {c.lower() for c in map(str, t.columns)}
        if any("date" in c for c in cols) and any(("version" in c) or ("booster" in c) or ("vehicle" in c) for c in cols):
            dfs.append(t)

    if not dfs:
        return pd.DataFrame()

    merged = pd.concat(dfs, ignore_index=True, sort=False)

    # Standardize columns
    merged.columns = merged.columns.map(lambda x: re.sub(r"\s+", " ", str(x)).strip().lower())

    # Try to extract flight number, booster version-ish string
    # Wikipedia does not always expose 'flight number'. We'll approximate by parsing a numeric sequence or using order.
    # We'll instead join by date (nearest day) when we merge. Keep version text column.
    candidate_cols = [c for c in merged.columns if any(k in c for k in ["version","booster","vehicle"])]
    if not candidate_cols:
        return pd.DataFrame()

    ver_col = candidate_cols[0]
    out = merged[[c for c in merged.columns if c in [ver_col, "date and time (utc)", "date", "date/time", "time (utc)"]]].copy()

    # Create a date column we can merge on (date only, UTC naive)
    def parse_date_any(s):
        try:
            return pd.to_datetime(s, errors="coerce", utc=True)
        except Exception:
            return pd.NaT

    if "date and time (utc)" in out.columns:
        out["dt"] = out["date and time (utc)"].apply(parse_date_any)
    elif "date/time" in out.columns:
        out["dt"] = out["date/time"].apply(parse_date_any)
    elif "date" in out.columns:
        out["dt"] = out["date"].apply(parse_date_any)
    else:
        out["dt"] = pd.NaT

    out = out.rename(columns={ver_col: "booster_version_raw"})
    out = out[['dt', 'booster_version_raw']].dropna(subset=['dt']).copy()

    # Simplify raw version strings into canonical labels
    def canonicalize_version(s: str) -> str:
        if not isinstance(s, str): return "Unknown"
        low = s.lower()
        if "block 5" in low or re.search(r"\bblk\.?\s*5\b", low):
            return "Block 5"
        if "full thrust" in low or "v1.2" in low or "ft" in low:
            return "FT"
        if "v1.1" in low or "v1. 1" in low:
            return "F9 v1.1"
        if "v1.0" in low or "v1" in low and "1.1" not in low and "1.2" not in low:
            return "F9 v1.0"
        # Sometimes the table shows simply "Falcon 9 Block 5", "Falcon 9 v1.1", etc.
        if "falcon 9" in low and "block 5" in low:
            return "Block 5"
        if "falcon 9" in low and "1.1" in low:
            return "F9 v1.1"
        if "falcon 9" in low and ("full thrust" in low or "ft" in low or "1.2" in low):
            return "FT"
        return "Unknown"

    out['booster_version'] = out['booster_version_raw'].apply(canonicalize_version)
    out['date'] = out['dt'].dt.date
    out = out[['date', 'booster_version']].drop_duplicates()

    return out

def fallback_version_by_date(dt: pd.Timestamp) -> str:
    if pd.isna(dt):
        return "Unknown"
    d = dt.tz_convert("UTC") if dt.tzinfo is not None else dt.tz_localize("UTC")
    d = d.date()
    if d <= pd.to_datetime("2013-09-28").date():
        return "F9 v1.0"
    if pd.to_datetime("2013-09-29").date() <= d <= pd.to_datetime("2015-12-21").date():
        return "F9 v1.1"
    if pd.to_datetime("2015-12-22").date() <= d <= pd.to_datetime("2018-05-10").date():
        return "FT"
    if d >= pd.to_datetime("2018-05-11").date():
        return "Block 5"
    return "Unknown"

lookup = fetch_booster_version_lookup()
print("Wikipedia lookup rows:", len(lookup))

# Merge on date-only first; if missing, apply fallback by date thresholds.
df['date'] = df['date_utc'].dt.date
df = df.merge(lookup, how='left', on='date', suffixes=('',''))

missing_mask = df['booster_version'].isna()
df.loc[missing_mask, 'booster_version'] = df.loc[missing_mask, 'date_utc'].apply(fallback_version_by_date)

# Persist lookup for auditing
if not lookup.empty:
    lookup.to_csv(DATA_DIR / "booster_version_lookup.csv", index=False)
else:
    # Create from unique dates in df for traceability
    tmp = df[['date','booster_version']].drop_duplicates().sort_values('date')
    tmp.to_csv(DATA_DIR / "booster_version_lookup.csv", index=False)

df['booster_version'] = df['booster_version'].fillna('Unknown')

df[['flight_number','date_utc','launch_site','orbit','payload_mass_kg','landing_outcome','booster_version']].head(8)


/var/folders/h7/0b1y_gfs7nz5k0qwkm0wfwp00000gn/T/ipykernel_13129/1063414728.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html.text)  # requires lxml installed


Wikipedia lookup rows: 27


,flight_number,date_utc,launch_site,orbit,payload_mass_kg,landing_outcome,booster_version
0,6,2010-06-04 18:45:00+00:00,CCSFS SLC 40,LEO,NaN,No attempt,F9 v1.0
1,7,2010-12-08 15:43:00+00:00,CCSFS SLC 40,LEO,NaN,No attempt,F9 v1.0
2,7,2010-12-08 15:43:00+00:00,CCSFS SLC 40,LEO,NaN,No attempt,F9 v1.0
3,8,2012-05-22 07:44:00+00:00,CCSFS SLC 40,LEO,525.0,No attempt,F9 v1.0
4,9,2012-10-08 00:35:00+00:00,CCSFS SLC 40,ISS,400.0,No attempt,F9 v1.0
5,9,2012-10-08 00:35:00+00:00,CCSFS SLC 40,LEO,400.0,No attempt,F9 v1.0
6,10,2013-03-01 19:10:00+00:00,CCSFS SLC 40,ISS,677.0,No attempt,F9 v1.0
7,11,2013-09-29 16:00:00+00:00,VAFB SLC 4E,PO,500.0,Failure,F9 v1.1


## Finalize tidy dataset

In [10]:

# Select a consistent column order for downstream tasks
cols_order = [
    'flight_number','date_utc','year','date','launch_site','site_region','site_locality','site_lat','site_lon',
    'rocket_name','booster_version','booster_block','core_serial',
    'launch_success','landing_outcome','landing_outcome_kind',
    'payload_id','payload_name','payload_mass_kg','orbit','customers','nationalities','is_nasa'
]

existing = [c for c in cols_order if c in df.columns]
df_final = df[existing].copy()

# Minor dedupe just in case
df_final = df_final.drop_duplicates(subset=['flight_number','payload_id','payload_name','orbit','launch_site','date_utc'])

# Save
OUT_CSV = DATA_DIR / "launches_clean.csv"
df_final.to_csv(OUT_CSV, index=False)

print("Saved:", OUT_CSV.resolve())
print("Rows:", len(df_final), "| Columns:", len(df_final.columns))
df_final.head(5)


Saved: /Users/johnpaulsandiego/Desktop/kData/data-science-capstone/data/launches_clean.csv
Rows: 192 | Columns: 23


,flight_number,date_utc,year,date,launch_site,site_region,site_locality,site_lat,site_lon,rocket_name,booster_version,booster_block,core_serial,launch_success,landing_outcome,landing_outcome_kind,payload_id,payload_name,payload_mass_kg,orbit,customers,nationalities,is_nasa
0,6,2010-06-04 18:45:00+00:00,2010,2010-06-04,CCSFS SLC 40,Florida,Cape Canaveral,28.561857,-80.577366,Falcon 9,F9 v1.0,1,B0003,True,No attempt,no_attempt,5eb0e4b7b6c3bb0006eeb1e7,Dragon Qualification Unit,NaN,LEO,SpaceX,United States,False
1,7,2010-12-08 15:43:00+00:00,2010,2010-12-08,CCSFS SLC 40,Florida,Cape Canaveral,28.561857,-80.577366,Falcon 9,F9 v1.0,1,B0004,True,No attempt,no_attempt,5eb0e4b9b6c3bb0006eeb1e8,COTS Demo Flight 1,NaN,LEO,NASA(COTS),United States,True
2,7,2010-12-08 15:43:00+00:00,2010,2010-12-08,CCSFS SLC 40,Florida,Cape Canaveral,28.561857,-80.577366,Falcon 9,F9 v1.0,1,B0004,True,No attempt,no_attempt,5eb0e4b9b6c3bb0006eeb1e9,Cubesats,NaN,LEO,NRO,NaN,False
3,8,2012-05-22 07:44:00+00:00,2012,2012-05-22,CCSFS SLC 40,Florida,Cape Canaveral,28.561857,-80.577366,Falcon 9,F9 v1.0,1,B0005,True,No attempt,no_attempt,5eb0e4bab6c3bb0006eeb1ea,COTS Demo Flight 2,525.0,LEO,NASA(COTS),United States,True
4,9,2012-10-08 00:35:00+00:00,2012,2012-10-08,CCSFS SLC 40,Florida,Cape Canaveral,28.561857,-80.577366,Falcon 9,F9 v1.0,1,B0006,True,No attempt,no_attempt,5eb0e4bab6c3bb0006eeb1eb,CRS-1,400.0,ISS,NASA (CRS),United States,True


## Quick QA checks against rubric needs

In [13]:

checks = {
    "has_orbit": 'orbit' in df_final.columns,
    "has_launch_site": 'launch_site' in df_final.columns,
    "has_payload_mass": 'payload_mass_kg' in df_final.columns,
    "has_year": 'year' in df_final.columns,
    "has_landing_outcome": 'landing_outcome' in df_final.columns,
    "has_booster_version": 'booster_version' in df_final.columns,
    "has_is_nasa": 'is_nasa' in df_final.columns,
}
print(checks)

print("\nUnique launch sites:", sorted(df_final['launch_site'].dropna().unique())[:10])
print("Unique orbits:", sorted(df_final['orbit'].dropna().unique())[:12])
print("Booster versions:", df_final['booster_version'].value_counts(dropna=False).head(10))


{'has_orbit': True, 'has_launch_site': True, 'has_payload_mass': True, 'has_year': True, 'has_landing_outcome': True, 'has_booster_version': True, 'has_is_nasa': True}

Unique launch sites: ['CCSFS SLC 40', 'KSC LC 39A', 'VAFB SLC 4E']
Unique orbits: ['ES-L1', 'GEO', 'GTO', 'HEO', 'ISS', 'LEO', 'MEO', 'PO', 'SO', 'SSO', 'TLI', 'VLEO']
Booster versions: booster_version
Block 5    133
FT          37
F9 v1.1     15
F9 v1.0      7
Name: count, dtype: int64



### How this dataset enables required outputs

- **Scatter plots** (Flight Number vs Launch Site / Payload vs Launch Site / Flight Number vs Orbit / Payload vs Orbit)  
  → Columns: `flight_number`, `launch_site`, `payload_mass_kg`, `orbit`.

- **Bar / Line charts** (Success Rate vs Orbit, Yearly Avg Success Rate)  
  → Columns: `launch_success`, `orbit`, `year`.

- **SQL queries** including:  
  - Unique launch sites → `launch_site`  
  - 5 records where site starts with "CCA" → `launch_site LIKE 'CCA%'`  
  - Total payload for **NASA** → `SUM(payload_mass_kg)` where `is_nasa = 1`  
  - Avg payload mass for **F9 v1.1** → `booster_version = 'F9 v1.1'`  
  - First successful **ground pad** landing → `landing_outcome LIKE 'Success (ground pad)%' ORDER BY date_utc`  
  - Successful **drone ship** landings with payload 4000–6000 → filter by `landing_outcome LIKE 'Success (drone ship)%'` and mass range  
  - Count successful vs failed → group by `launch_success`  
  - Booster(s) with max payload → group by `core_serial` / `payload_mass_kg`  
  - Failed **drone ship** landings in 2015 with booster versions & launch sites → filters on `year`=2015, outcome, with `booster_version` & `launch_site`  
  - Rank landing outcomes between dates → filter by `date_utc` range and group by `landing_outcome`

- **Folium maps** → `launch_site`, `site_lat`, `site_lon`, `launch_success`, `landing_outcome`.

- **Dash** → interactive filters over `orbit`, `payload_mass_kg`, `launch_site`, `booster_version`, and `landing_outcome`.

- **ML** → label: `launch_success`; features: `orbit`, `payload_mass_kg`, `booster_block`, `booster_version`, `launch_site`, etc.
